In [ ]:
#Import libraries and packages

import os
import time

In [16]:
#Size of the file - 5.6GB
os.path.getsize('C:/Users/Oludipe j/Documents/Data Analysis/Data Glacier/2019-Oct.csv')

5668612855

In [17]:
#Read in the data with Dask

from dask import dataframe as dd
start = time.time()
dask_df = dd.read_csv('C:/Users/Oludipe j/Documents/Data Analysis/Data Glacier/2019-Oct.csv')
end = time.time()
print("Read csv with dask: ",(end-start),"sec")



Read csv with dask:  0.00844883918762207 sec


In [7]:
#Read in the data with Pandas

import pandas as pd
start = time.time()
panda_df = pd.read_csv('C:/Users/Oludipe j/Documents/Data Analysis/Data Glacier/2019-Oct.csv')
end = time.time()
print("Read csv with pandas: ",(end-start),"sec")


Read csv with pandas:  66.20829653739929 sec


In [ ]:
#Read in the data with Modin and Ray

import modin.pandas as pd
import ray
ray.shutdown()
ray.init()
start = time.time()
df = pd.read_csv('C:/Users/Oludipe j/Documents/Data Analysis/Data Glacier/2019-Oct.csv')
end = time.time()
print("Read csv with modin and ray: ",(end-start),"sec")


In [ ]:
#Results show that Dask is better than Pandas, Modin and Ray, with the least reading time of 0.0084 sec

In [18]:
#Using Dask
from dask import dataframe as dd
df = dd.read_csv('C:/Users/Oludipe j/Documents/Data Analysis/Data Glacier/2019-Oct.csv', delimiter = ',')

In [19]:
#View dataframe info
df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 9 entries, event_time to user_session
dtypes: object(5), float64(1), int64(3)

In [28]:
#View table
df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,NaN,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d


In [29]:
#Remove special character
df.columns=df.columns.str.replace('[#,@,&]','')

In [30]:
#View column data
data = df.columns
data

Index(['event_time', 'event_type', 'product_id', 'category_id',
       'category_code', 'brand', 'price', 'user_id', 'user_session'],
      dtype='object')

In [31]:
#Import more libraries and packages

import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re

In [32]:
%%writefile utility.py

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.load(stream, Loader=yaml.Loader)
        except yaml.YAMLError as exc:
            logging.error(exc)

def col_header_val(df,table_config):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Writing utility.py


In [33]:
%%writefile store.yaml
file_type: csv
dataset_name: file
file_name: Rate
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
      - event_time
      - event_type 
      - product_id
      - category_id
      - category_code
      - brand
      - price
      - user_id
      - user_session


Writing store.yaml


In [48]:
# Reading yaml file
import yaml

with open("store.yaml", "r") as stream:
    try:
        print(yaml.safe_load(stream))
    except yaml.YAMLError as exc:
        print(exc)

{'file_type': 'csv', 'dataset_name': 'file', 'file_name': 'Rate', 'table_name': 'edsurv', 'inbound_delimiter': ',', 'outbound_delimiter': '|', 'skip_leading_rows': 1, 'columns': ['event_time', 'event_type', 'product_id', 'category_id', 'category_code', 'brand', 'price', 'user_id', 'user_session']}


In [45]:
# Reading process of the file using Dask
from dask import dataframe as dd
df_sample = dd.read_csv('C:/Users/Oludipe j/Documents/Data Analysis/Data Glacier/2019-Oct.csv',delimiter=',')
df_sample.head()



,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,NaN,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d


In [71]:
#Number of columns
print("Number of rows:", df.shape[0].compute())

Number of rows: 42448764


In [64]:
#Number of columns
print("Number of columns:", df.shape[1])

Number of columns: 9


In [75]:
df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,NaN,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d


In [ ]:
import datetime
import csv
import gzip

from dask import dataframe as dd
df = dd.read_csv('C:/Users/Oludipe j/Documents/Data Analysis/Data Glacier/2019-Oct.csv',delimiter=',')

# Write csv in gz format in pipe separated text file (|)
df.to_csv('Rate.csv.gz',
          sep='|',
          header=True,
          index=False,
          quoting=csv.QUOTE_ALL,
          compression='gzip',
          quotechar='"',
          doublequote=True,
          lineterminator='\n')

In [ ]:
#number of files in gz format folder
import os
entries = os.listdir('C:/Users/Oludipe j/Documents/Data Analysis/Data Glacier/2019-Oct.csv.gz/')
for entry in entries:
    print(entry)


In [ ]:
#size of the gz format folder
os.path.getsize('C:/Users/Oludipe j/Documents/Data Analysis/Data Glacier/2019-Oct.csv.gz')

